# Метрики качества классификации

###### Введение
В задачах классификации может быть много особенностей, влияющих на подсчет качества: различные цены ошибок, несбалансированность классов и т.д. Из-за этого существует большое количество метрик качества — каждая из них рассчитана на определенное сочетание свойств задачи и требований к ее решению.

Меры качества классификации можно разбить на две большие группы: предназначенные для алгоритмов, выдающих номера классов, и для алгоритмов, выдающих оценки принадлежности к классам. К первой группе относятся доля правильных ответов, точность, полнота, F-мера. Ко второй — площади под ROC- или PR-кривой.

In [38]:
import pandas as pd
import numpy as np

Загрузите файл classification.csv. В нем записаны истинные классы объектов выборки (колонка true) и ответы некоторого классификатора (колонка pred).

In [2]:
df = pd.read_csv('classification.csv')

In [5]:
df.head()

,true,pred
0,1,0
1,1,1
2,1,1
3,0,0
4,1,1


Заполните таблицу ошибок классификации:    
Actual Positive	Actual Negative  
Predicted Positive	TP	FP  
Predicted Negative	FN	TN  
Для этого подсчитайте величины TP, FP, FN и TN согласно их определениям. Например, FP — это количество объектов, имеющих класс 0, но отнесенных алгоритмом к классу 1. Ответ в данном вопросе — четыре числа через пробел.

In [61]:
tp = sum(df[df['true'] == 1]['pred'] == 1)
fp = sum(df[df['true'] == 0]['pred'] == 1)
fn = sum(df[df['true'] == 1]['pred'] == 0)
tn = sum(df[df['true'] == 0]['pred'] == 0)
print(tp,fp,fn,tn)

43 34 59 64


Посчитайте основные метрики качества классификатора:

- Accuracy (доля верно угаданных) — sklearn.metrics.accuracy_score
- Precision (точность) — sklearn.metrics.precision_score
- Recall (полнота) — sklearn.metrics.recall_score
- F-мера — sklearn.metrics.f1_score

In [15]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [56]:
print(round(accuracy_score(df['true'],df['pred']),2))
print(round(precision_score(df['true'],df['pred']),2))
print(round(recall_score(df['true'],df['pred']),2))
print(round(f1_score(df['true'],df['pred']),2))

0.54
0.56
0.42
0.48


Имеется четыре обученных классификатора. В файле scores.csv записаны истинные классы и значения степени принадлежности положительному классу для каждого классификатора на некоторой выборке:

- для логистической регрессии — вероятность положительного класса (колонка score_logreg),
- для SVM — отступ от разделяющей поверхности (колонка score_svm),
- для метрического алгоритма — взвешенная сумма классов соседей (колонка score_knn),
- для решающего дерева — доля положительных объектов в листе (колонка score_tree).

In [17]:
data = pd.read_csv('scores.csv')

In [19]:
data.head()

,true,score_logreg,score_svm,score_knn,score_tree
0,0,0.683832,0.145976,0.787063,0.500000
1,1,0.801966,0.239511,1.000000,0.833333
2,0,0.382315,-0.245701,0.000000,0.000000
3,1,0.506797,-0.137058,0.000000,0.105263
4,1,0.488781,-0.154148,0.000000,0.105263


Посчитайте площадь под ROC-кривой для каждого классификатора. Какой классификатор имеет наибольшее значение метрики AUC-ROC (укажите название столбца)? Воспользуйтесь функцией sklearn.metrics.roc_auc_score.

In [20]:
from sklearn.metrics import roc_auc_score

In [27]:
roc_max = 0
c = 0
for clf in range(1,5):
    if roc_auc_score(data['true'], data.iloc[:,clf]) > roc_max:
        roc_max = roc_auc_score(data['true'], data.iloc[:,clf])
        c = clf
print(data.columns[c])
print(roc_max)

score_logreg
0.719187675070028


Какой классификатор достигает наибольшей точности (Precision) при полноте (Recall) не менее 70% ? 

Чтобы получить ответ на этот вопрос, найдите все точки precision-recall-кривой с помощью функции sklearn.metrics.precision_recall_curve. Она возвращает три массива: precision, recall, thresholds. В них записаны точность и полнота при определенных порогах, указанных в массиве thresholds. Найдите максимальной значение точности среди тех записей, для которых полнота не меньше, чем 0.7.

In [28]:
from sklearn.metrics import precision_recall_curve

In [51]:
r_max = 0
c = 0
for clf in range(1,5):    
    d = np.transpose(pd.DataFrame(precision_recall_curve(data['true'], data.iloc[:,clf])))
    d.columns=['precision', 'recall', 'thresholds']
    r = d[d['recall'] >= 0.7]['precision'].max()
    if r > r_max:
        r_max = r
        c = clf
print(data.columns[c])
print(r_max)

score_tree
0.6517857142857143
